In [1]:
from datasets import list_datasets
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from IPython import get_ipython
import pandas as pd
from tqdm import tqdm
from models.MergeModel import MergeModel

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', 2)

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartForSequenceClassification

In [3]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [4]:
vocab_size = tokenizer.vocab_size
vocab_size

50265

In [5]:
df = pd.read_csv("dataset/test.csv")

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)

Number of training sentences: 11,490



,article,highlights,id
1158,The force wasn't with everyone during the Star...,Mark and Carrie are reprising their roles as L...,01dfc3575d8af0e6a10126901ddb357f933821cf
10934,Memphis moved into a tie for the second seed i...,Memphis Grizzles' defence helped them to a 100...,f25746182409e4c04a30c2919225b167cdb0856e
2464,This is the shocking dashcam footage which sho...,Shaun Ingram booked his Ford Focus ST for an M...,21182fce80da04843a9a615e2f08d873dcd090f4
7757,The man who threw a banana peel at Dave Chappe...,"Christian Englander, 30, threw a banana peel a...",a37dda8b1c1ad8c8628c3b2a29a02656345bee28
4249,"Known for his life of fast cars, supermodels, ...","Dan Bilzerian, 34, arrested on explosives char...",4cfe02fe65eedf9364dd2b9633dca27349ec02aa
6425,"YouTube is set to offer an ad free, subscripti...",Plan was revealed in an email sent out to YouT...,826dd7014f3a2a06281f4f0147ad2203c6372fc4
1354,The world's youngest DJ who is just two years ...,Two-year-old Oratilwe Hlongwane can wow fans b...,060cc51c4ae6c738da32dc9e6f0b2f354c97a4f0
9358,Albert Park - Australian Grand Prix . Sepang -...,Round three of the Formula One season takes pl...,cac57b5a493ce89c6752d2ee92e2340bd8805c42
2937,Straight after Easter on planet football comes...,Chelsea boss Jose Mourinho has spoken in defen...,2d2aa19b5c53bf967a8a53ee3e64e0bbc3d58d0a
2445,Jonjo O'Neill hosted a media stable visit on T...,Crabbie's Grand National takes place at Aintre...,20b231e56a0a2104a96e167659222986fbcaeddd


In [6]:
train_sentence = df.article.values
train_target = df.highlights.values

In [7]:
num_data_points = 10
train_sentence = list(train_sentence[:num_data_points])
train_target = list(train_target[:num_data_points])

In [8]:
article_encoding = tokenizer(train_sentence, return_tensors='pt', padding=True, truncation = True)
summary_encoding = tokenizer(train_target, return_tensors='pt', padding=True, truncation = True)

In [9]:
article_input_ids = article_encoding['input_ids']
article_attention_mask = article_encoding['attention_mask']

In [10]:
summary_input_ids = summary_encoding['input_ids']
summary_attention_mask = summary_encoding['attention_mask']

In [11]:
print(article_input_ids.shape, article_attention_mask.shape)

torch.Size([10, 1024]) torch.Size([10, 1024])


In [12]:
print(summary_input_ids.shape, summary_attention_mask.shape)

torch.Size([10, 57]) torch.Size([10, 57])


In [13]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [14]:
batch_size = 1
train_data = TensorDataset(article_input_ids, article_attention_mask,\
                           summary_input_ids, summary_attention_mask)

In [15]:
train_sampler = RandomSampler(train_data)

In [16]:
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [17]:
def loss_fn(lm_logits, labels):
    loss_fct = CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
    loss = loss_fct(lm_logits.view(-1, vocab_size), labels.view(-1))
    return loss

In [18]:
mse_loss = nn.MSELoss()

In [19]:
class Temp(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return 0

In [20]:
# -------------- Testing Scatter -----------

In [21]:
a = torch.randint(0,10, (2,3,3))
a, a.shape

(tensor([[[8, 6, 9],
          [2, 8, 2],
          [7, 2, 6]],
 
         [[5, 7, 3],
          [5, 3, 4],
          [7, 3, 9]]]),
 torch.Size([2, 3, 3]))

In [22]:
idx =  torch.argmax(a, dim=2, keepdims=  True)

In [23]:
mask = torch.zeros_like(a).scatter_(2, idx, 1.)

In [24]:
(a * mask)

tensor([[[0, 0, 9],
         [0, 8, 0],
         [7, 0, 0]],

        [[0, 7, 0],
         [5, 0, 0],
         [0, 0, 9]]])

In [25]:
(a * mask).sum(axis=2).long().type()

'torch.LongTensor'

In [26]:
# --------------- End ------------------------

In [27]:
summary_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [28]:
senti_model = BartForSequenceClassification.from_pretrained('facebook/bart-base')

Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartForSequenceClassification: ['final_logits_bias']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [29]:
summary_model = summary_model.cuda()
senti_model = senti_model.cuda()

In [30]:
merge_model = MergeModel(summary_model, senti_model)

In [31]:
optimizer = torch.optim.SGD(merge_model.parameters(),1e-3,momentum=0.9,weight_decay=0.01)

In [32]:
epochs = 500

for eps in tqdm(range(epochs)):
#     print('Epoch: ', eps)
    epoch_loss = 0
    for step, batch in enumerate(train_dataloader):
        
        batch[0] = batch[0].cuda()
        batch[1] = batch[1].cuda()
        batch[2] = batch[2].cuda()
        batch[3] = batch[3].cuda()
        
        summary_model.train()
        
        optimizer.zero_grad()
        
        summary_out,*sentiments = merge_model(batch[0],batch[1], batch[2], batch[3])
        
        loss1 = loss_fn(summary_out.logits, batch[2])
        loss2 = mse_loss(sentiments[0].logits, sentiments[1].logits)
        
        final_loss = loss1 + loss2
        
        epoch_loss = epoch_loss + final_loss
        
        final_loss.backward()
        optimizer.step()
    
    if not eps%10:
        print("Epoch: %d, Loss: %.3f" % (eps, epoch_loss.item()))


  0%|          | 1/500 [00:06<53:51,  6.48s/it]

Epoch: 0, Loss: 83.830


  2%|▏         | 11/500 [01:12<52:50,  6.48s/it]

Epoch: 10, Loss: 51.270


  4%|▍         | 21/500 [02:18<53:18,  6.68s/it]

...

 96%|█████████▌| 481/500 [51:06<02:00,  6.33s/it]

Epoch: 480, Loss: 0.118


 98%|█████████▊| 491/500 [52:11<00:56,  6.29s/it]

Epoch: 490, Loss: 0.127


100%|██████████| 500/500 [53:09<00:00,  6.38s/it]


# Testing

In [33]:
b1,b2,b3,b4 = next(iter(train_dataloader))
b1 = b1.cuda()
b2 = b2.cuda()
b3 = b3.cuda()
b4 = b4.cuda()

In [34]:
summary_out,*sentiments= merge_model(b1, b2, b3, b4)

In [35]:
tokenizer.decode(list(b1.view(-1)))

'<s>(CNN)Marco Rubio is all in.  The Republican senator from Florida has announced that he is seeking the Republican presidential nomination, running on an optimistic message that he embodies the promise of the American Dream. With his youthful energy and Hispanic roots, it\'s tempting to see Rubio as the new blood that the GOP needs in order to compete against Hillary Clinton in 2016.  Yet Rubio has been his own worst enemy on what could have been his two signature issues: immigration reform and Cuba relations.  He holds little appeal to Latino voters.  And unless he can offer new ideas, his climb to the Republican nomination will be steep. Back in 2013, Rubio was a member of the Senate "Gang of 8" that crafted a bipartisan proposal for comprehensive reform, including a path to citizenship for the nation\'s estimated 11 million undocumented immigrants.  He later distanced himself from the bill after it ran into resistance from House Republicans, and now says he favors a piecemeal appr

In [36]:
tokenizer.decode(list(summary_out.logits.argmax(2).view(-1)))

"<s>Raul Reyes: In seeking Latino vote, Marco Rubio his own worst enemy on two key issues: immigration reform, Cuba relations.\nHe says on health care, climate change and other issues, he breaks from Latinos' positions. Polls show they don't favor him.</s>"